In [2]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [1]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (181 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123668 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed

In [2]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [3]:
GROQ_API_KEY = "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"

In [4]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [5]:
loader = DirectoryLoader("data/", glob="./*.pdf", loader_cls=UnstructuredPDFLoader)
documents = loader.load()

In [6]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [7]:
persist_directory = "doc_db"

In [8]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [10]:
retriever = vectorstore.as_retriever()

In [11]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [13]:
query = "What are Directed Acyclic Graphs?"
response = qa_chain.invoke({"query":query})

In [14]:
print(response)

{'query': 'What are Directed Acyclic Graphs?', 'result': 'Directed Acyclic Graphs (DAGs) are a type of graph that is both directed and acyclic. \n\n- Directed: In a directed graph, edges have a direction or orientation, meaning that they can only be traversed in one direction. This is in contrast to undirected graphs, where edges do not have a direction.\n\n- Acyclic: An acyclic graph is a graph that does not contain any cycles. A cycle is a path in the graph that starts and ends at the same vertex, and passes through at least one edge more than once. In other words, a cycle is a path that "loops back" on itself.\n\nSo, a DAG is a graph that has edges with direction and does not have any cycles. This means that it is possible to traverse the graph in a linear fashion, from one vertex to another, without ever repeating a path.', 'source_documents': [Document(metadata={'source': 'data/2.pdf'}, page_content='Euler Walk An Euler Walk (sometimes called an Euler Path) is a walk that goes thr

In [15]:
print(response["result"])

Directed Acyclic Graphs (DAGs) are a type of graph that is both directed and acyclic. 

- Directed: In a directed graph, edges have a direction or orientation, meaning that they can only be traversed in one direction. This is in contrast to undirected graphs, where edges do not have a direction.

- Acyclic: An acyclic graph is a graph that does not contain any cycles. A cycle is a path in the graph that starts and ends at the same vertex, and passes through at least one edge more than once. In other words, a cycle is a path that "loops back" on itself.

So, a DAG is a graph that has edges with direction and does not have any cycles. This means that it is possible to traverse the graph in a linear fashion, from one vertex to another, without ever repeating a path.
